In [1]:


import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 
from sklearn import metrics

import os


INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_rotation_x_",
    "body_rotation_y_",
    "body_rotation_z_",
#     "body_attitude_x_",
#     "body_attitude_y_",
#     "body_attitude_z_",
    "body_gravity_x_",
    "body_gravity_y_",
    "body_gravity_z_"
]

# INPUT_SIGNAL_TYPES = [
#     "body_outbound_acc_x_",
#     "body_outbound_acc_y_",
#     "body_outbound_acc_z_",
#     "body_outbound_rotation_x_",
#     "body_outbound_rotation_y_",
#     "body_outbound_rotation_z_",
#     "body_outbound_gravity_x_",
#     "body_outbound_gravity_y_",
#     "body_outbound_gravity_z_"
#     "body_return_acc_x_",
#     "body_return_acc_y_",
#     "body_return_acc_z_",
#     "body_return_rotation_x_",
#     "body_return_rotation_y_",
#     "body_return_rotation_z_",
#     "body_return_gravity_x_",
#     "body_return_gravity_y_",
#     "body_return_gravity_z_"
# ]

# Output classes
LABELS = [
    "FALSE",
    "TRUE" 
] 



DATASET_PATH = "/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_train/"


DATASET_FINAL_TEST_PATH = "/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_test/"


print("\n" + "Dataset is now located at: " + DATASET_PATH)



Dataset is now located at: /home/soon2soon/Notebooks/DreamChallenge/data/processed_data_train/


## Preparing dataset:

In [2]:
TRAIN = "train/"
TEST = "test/"


def load_X(X_signals_paths):
    X_signals = []
    
    for signal_type_path in X_signals_paths:
        file = open(signal_type_path, 'r')
        print(signal_type_path)
        
        X_signals.append(
            [np.array(serie, dtype=np.float32) for serie in [
                row.replace('  ', ' ').strip().split(' ') for row in file
                
            ]]
        )
        
        file.close()

    
    return np.transpose(np.array(X_signals), (1, 2, 0))

X_train_signals_paths = [
    DATASET_PATH + TRAIN + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
]


X_test_signals_paths = [
    DATASET_PATH + TEST + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]

X_test_final_signals_paths = [
    DATASET_FINAL_TEST_PATH + 'test/' + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]



X_train = load_X(X_train_signals_paths)

X_test = load_X(X_test_signals_paths)

# X_test_final = load_X(X_test_final_signals_paths)



def load_y(y_path):
    file = open(y_path, 'r')
    print(y_path)
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]], 
        dtype=np.int32
    )
    file.close()
    
    return y_

y_train_path = DATASET_PATH + TRAIN + "y_train.txt"

y_test_path = DATASET_PATH + TEST + "y_test.txt"

# y_test_final_path = DATASET_FINAL_TEST_PATH + TEST + "y_test.txt"



y_train = load_y(y_train_path)

y_test = load_y(y_test_path)

# y_test_final = load_y(y_test_final_path)




print("Dataset load done.")


/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_train/train/body_acc_x_train.txt
/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_train/train/body_acc_y_train.txt
/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_train/train/body_acc_z_train.txt
/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_train/train/body_rotation_x_train.txt
/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_train/train/body_rotation_y_train.txt
/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_train/train/body_rotation_z_train.txt
/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_train/train/body_gravity_x_train.txt
/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_train/train/body_gravity_y_train.txt
/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_train/train/body_gravity_z_train.txt
/home/soon2soon/Notebooks/DreamChallenge/data/processed_data_train/test/body_acc_x_test.txt
/home/soon2soon/Notebooks/DreamChal

## Additionnal Parameters:


In [3]:
# Input Data 

training_data_count = len(X_train)  # training series (with 50% overlap between each serie)


print("+ training_data_count",training_data_count )

test_data_count = len(X_test)  # number of testing series
n_steps = len(X_train[0])  # timesteps per series
n_input = len(X_train[0][0])  # number of input parameters per timestep

print("+ test_data_count",test_data_count )

print("+ n_steps", n_steps)
print("+ n_input", n_input)



n_hidden = 96 # Hidden layer num of features
n_classes = 2 
n_hidden_3 = 32

# Training parameters

learning_rate = 0.001
lambda_loss_amount = 0.0015
training_iters = training_data_count * 20  # Loop xx times on the dataset
batch_size = 300
display_iter = 5000  # To show test set accuracy during training


print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))


+ training_data_count 189434
+ test_data_count 3973
+ n_steps 512
+ n_input 9
(X shape, y shape, every X's mean, every X's standard deviation)
(3973, 512, 9) (3973, 1) 0.0266799 0.65206


## Utility functions for training:

In [4]:
def BiRNN(x, weights, biases):

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, num_input)
    x = tf.unstack(x, n_steps, 1)

    # Forward direction cell
    lstm_fw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    
    lstm_fw_cell = tf.contrib.rnn.DropoutWrapper(lstm_fw_cell, output_keep_prob=0.7) 
    
    # Backward direction cell
    lstm_bw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    
    lstm_bw_cell = tf.contrib.rnn.DropoutWrapper(lstm_bw_cell, output_keep_prob=0.7) 

    # Get lstm cell output
    
    outputs, _, _ = tf.contrib.rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x, dtype=tf.float32)


    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

# def BiRNN_attn(x, weights, biases):

#     # Prepare data shape to match `rnn` function requirements
#     # Current data input shape: (batch_size, timesteps, n_input)
#     # Required shape: 'timesteps' tensors list of shape (batch_size, num_input)

#     # Unstack to get a list of 'timesteps' tensors of shape (batch_size, num_input)
#     x = tf.unstack(x, n_steps, 1)

#     # Define lstm cells with tensorflow
#     # Forward direction cell
#     lstm_fw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    
#     drop = tf.contrib.rnn.DropoutWrapper(lstm_fw_cell, input_keep_prob = 0.7)
    
#     lstm_fw_cell = tf.contrib.rnn.AttentionCellWrapper(drop, attn_length=4, state_is_tuple = True)

    
#     # Backward direction cell
#     lstm_bw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    
#     drop = tf.contrib.rnn.DropoutWrapper(lstm_bw_cell, input_keep_prob = 0.7)
    
#     lstm_bw_cell = tf.contrib.rnn.AttentionCellWrapper(drop, attn_length=4, state_is_tuple = True)
    
    
#     # Get lstm cell output
    
#     outputs, _, _ = tf.contrib.rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x, dtype=tf.float32)


#     # Linear activation, using rnn inner loop last output
#     return tf.matmul(outputs[-1], weights['out']) + biases['out']


def LSTM_RNN_3(_X, _weights, _biases):
    # Reshape to split input to 3 channel List
    _X = tf.transpose(_X, [2,0,1])
    _X = tf.unstack(_X)
    
    _X_acc = tf.stack(_X[:3])
    _X_rot = tf.stack(_X[3:6])
    _X_gra = tf.stack(_X[6:])
    # New shape: (3, batch_size, n_steps)
    
    _X_acc = tf.transpose(_X_acc, [2, 1, 0])
    _X_rot = tf.transpose(_X_rot, [2, 1, 0])
    _X_gra = tf.transpose(_X_gra, [2, 1, 0])

    # Reshape to prepare input to hidden activation
    _X_acc = tf.reshape(_X_acc, [-1, 3])
    _X_rot = tf.reshape(_X_rot, [-1, 3])
    _X_gra = tf.reshape(_X_gra, [-1, 3])
    # new shape: (3*batch_size, n_input)

    # Linear activation
    _X_acc = tf.nn.relu(tf.matmul(_X_acc, _weights['hidden_acc']) + _biases['hidden_acc'])
    _X_rot = tf.nn.relu(tf.matmul(_X_rot, _weights['hidden_rot']) + _biases['hidden_rot'])
    _X_gra = tf.nn.relu(tf.matmul(_X_gra, _weights['hidden_gra']) + _biases['hidden_gra'])
    
    _X = tf.concat([_X_acc, _X_rot, _X_gra],1)
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(_X, n_steps, 0)
    # new shape: n_steps * (batch_size, n_hidden)


    # Define two stacked LSTM cells (two recurrent layers deep) with tensorflow
    lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    
    lstm_cell_1 = tf.contrib.rnn.DropoutWrapper(lstm_cell_1, input_keep_prob=0.7, output_keep_prob=0.7)
    
    lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    
    lstm_cell_2 = tf.contrib.rnn.DropoutWrapper(lstm_cell_2, input_keep_prob=0.7, output_keep_prob=0.7) 
    
        
    lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], state_is_tuple=True)
    


    # Get LSTM cell output
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X, dtype=tf.float32)
    

    # Get last time step's output feature for a "many to one" style classifier
    lstm_last_output = outputs[-1]
    
    # Linear activation
    return tf.matmul(lstm_last_output, _weights['out']) + _biases['out'], lstm_last_output

def LSTM_RNN(_X, _weights, _biases):
    
    # input shape: (batch_size, n_steps, n_input)
    _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
    # Reshape to prepare input to hidden activation
    _X = tf.reshape(_X, [-1, n_input]) 
    # new shape: (n_steps*batch_size, n_input)
    
    # Linear activation
    _X = tf.nn.relu(tf.matmul(_X, _weights['hidden']) + _biases['hidden'])
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(_X, n_steps, 0) 
    # new shape: n_steps * (batch_size, n_hidden)

    # Define two stacked LSTM cells (two recurrent layers deep) with tensorflow
    lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    
    lstm_cell_1 = tf.contrib.rnn.DropoutWrapper(lstm_cell_1, input_keep_prob=0.7, output_keep_prob=0.7)
    
    lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    
    lstm_cell_2 = tf.contrib.rnn.DropoutWrapper(lstm_cell_2, input_keep_prob=0.7, output_keep_prob=0.7) 
    
        
    lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], state_is_tuple=True)
    


    # Get LSTM cell output
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X, dtype=tf.float32)
    

    # Get last time step's output feature for a "many to one" style classifier
    lstm_last_output = outputs[-1]
    
    # Linear activation
    return tf.matmul(lstm_last_output, _weights['out']) + _biases['out'], lstm_last_output
 

def extract_batch_size(_train, step, batch_size):
    # Function to fetch a "batch_size" amount of data from "(X|y)_train" data. 
    
    shape = list(_train.shape)
    shape[0] = batch_size
    batch_s = np.empty(shape)

    for i in range(batch_size):
        # Loop index
        index = ((step-1)*batch_size + i) % len(_train)
        batch_s[i] = _train[index] 

    return batch_s


def one_hot(y_):
    # Function to encode output labels from number indexes 
    # e.g.: [[5], [0], [3]] --> [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
    
    y_ = y_.reshape(len(y_))
      
    n_values = int(np.max(y_)) + 1
    
    if n_values == 1:
        n_values = 2
    
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]  # Returns FLOATS


In [5]:

# Graph input/output
x = tf.placeholder(tf.float32, [None, n_steps, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])


# Graph weights
weights = {
    'hidden': tf.Variable(tf.random_normal([n_input, n_hidden])), # Hidden layer weights
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes], mean=1.0))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

weights_3 = {
    'hidden_acc': tf.Variable(tf.random_normal([3, n_hidden_3])), # Hidden layer weights
    'hidden_rot': tf.Variable(tf.random_normal([3, n_hidden_3])),
    'hidden_gra': tf.Variable(tf.random_normal([3, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes], mean=1.0))
}
biases_3 = {
    'hidden_acc': tf.Variable(tf.random_normal([n_hidden_3])),
    'hidden_rot': tf.Variable(tf.random_normal([n_hidden_3])),
    'hidden_gra': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# pred = LSTM_RNN_3(x, weights, biases)
# pred, last_state = LSTM_RNN_3(x, weights_3, biases_3)
pred, last_state = LSTM_RNN(x, weights, biases)
# pred = BiRNN(x, weights_bi, biases)
# pred = BiRNN_attn(x, weights_bi, biases) 



# Loss, optimizer and evaluation
l2 = lambda_loss_amount * sum(
    tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()
) # L2 loss prevents this overkill neural network to overfit the data

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred)) + l2 # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer
# optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost) # RMS Optimizer

correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


## Train the neural network:

In [6]:
# To keep track of training's performance
test_losses = []
test_accuracies = []
train_losses = []
train_accuracies = []

# Launch the graph
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)




saver = tf.train.Saver()
model_path = "/home/soon2soon/Notebooks/DreamChallenge/LSTM-Human-Activity-Recognition/tmp/model.ckpt"



# Perform Training steps with "batch_size" amount of example data at each loop
step = 1
while step * batch_size <= training_iters:
    batch_xs =         extract_batch_size(X_train, step, batch_size)
    batch_ys = one_hot(extract_batch_size(y_train, step, batch_size))

    # Fit training using batch data
    _, loss, acc = sess.run(
        [optimizer, cost, accuracy],
        feed_dict={
            x: batch_xs, 
            y: batch_ys
        }
    )
    train_losses.append(loss)
    train_accuracies.append(acc)
    
    # Evaluate network only at some steps for faster training: 
    if (step*batch_size % display_iter == 0) or (step == 1) or (step * batch_size > training_iters):
        
        # To not spam console, show training accuracy/loss in this "if"
        print("Training iter #" + str(step*batch_size) + \
              ":   Batch Loss = " + "{:.6f}".format(loss) + \
              ", Accuracy = {}".format(acc))
        
        
        
        
        # Evaluation on the test set (no learning made here - just evaluation for diagnosis)
        loss, acc = sess.run(
            [cost, accuracy], 
            feed_dict={
                x: X_test,
                y: one_hot(y_test)
            }
        )
        test_losses.append(loss)
        test_accuracies.append(acc)
        print("PERFORMANCE ON TEST SET: " + \
              "Batch Loss = {}".format(loss) + \
              ", Accuracy = {}".format(acc))
        
        
        

    step += 1

print("Optimization Finished!")

save_path = saver.save(sess, model_path)
print("Model saved in file: %s" % save_path)

# Accuracy for test data





# one_hot_predictions, accuracy, final_loss, l_h = sess.run(
#     [pred, accuracy, cost, last_state],
#     feed_dict={
#         x: X_test,
#         y: one_hot(y_test)
#     }
# )


# print("FINAL RESULT: " + \
#       "Batch Loss = {}".format(final_loss) + \
#       ", Accuracy = {}".format(accuracy))


# np.savetxt("output_hidden_test.txt", l_h)

# print("hidden test write donw")
#####################


# itter = 1
# chunk_size = 500
# while itter * chunk_size <= len(X_test_final):
#     batch_xs =         extract_batch_size(X_test_final, step, chunk_size)
#     batch_ys = one_hot(extract_batch_size(y_test_final, step, chunk_size))

#     # Fit training using batch data

#     a_t, _, l_h_t = sess.run(
#         [accuracy, cost, last_state],
#         feed_dict={
#             x: batch_xs,
#             y: batch_ys
#         }
#     )

#     print("FINAL RESULT: " + \
#           ", Accuracy = {}".format(a_t))

#     np.savetxt('output_state_result/'+"output_hidden_train"+str(itter)+".txt", l_h_t)

#     print(itter, "hidden test final write donw!")
#     itter= itter+1
    
# print("iter done.")






Training iter #300:   Batch Loss = 2.760908, Accuracy = 0.7033332586288452
PERFORMANCE ON TEST SET: Batch Loss = 2.9690566062927246, Accuracy = 0.5502139925956726
Training iter #15000:   Batch Loss = 2.505090, Accuracy = 0.6399999856948853
PERFORMANCE ON TEST SET: Batch Loss = 2.533428430557251, Accuracy = 0.5572615265846252
Training iter #30000:   Batch Loss = 2.428706, Accuracy = 0.5066666603088379
PERFORMANCE ON TEST SET: Batch Loss = 2.3904006481170654, Accuracy = 0.5512207746505737
Training iter #45000:   Batch Loss = 2.219393, Accuracy = 0.6633332967758179
PERFORMANCE ON TEST SET: Batch Loss = 2.2493796348571777, Accuracy = 0.6204379796981812
Training iter #60000:   Batch Loss = 2.274346, Accuracy = 0.5966666340827942
PERFORMANCE ON TEST SET: Batch Loss = 2.1990041732788086, Accuracy = 0.624213457107544
Training iter #75000:   Batch Loss = 2.004452, Accuracy = 0.6600000262260437
PERFORMANCE ON TEST SET: Batch Loss = 2.027860164642334, Accuracy = 0.6219481825828552
Training iter #

Training iter #750000:   Batch Loss = 0.716459, Accuracy = 0.653333306312561
PERFORMANCE ON TEST SET: Batch Loss = 0.7258167266845703, Accuracy = 0.6274855136871338
Training iter #765000:   Batch Loss = 0.682209, Accuracy = 0.7066666483879089
PERFORMANCE ON TEST SET: Batch Loss = 0.7328992486000061, Accuracy = 0.6219481825828552
Training iter #780000:   Batch Loss = 0.707365, Accuracy = 0.6399999856948853
PERFORMANCE ON TEST SET: Batch Loss = 0.7204412817955017, Accuracy = 0.6209414005279541
Training iter #795000:   Batch Loss = 0.588446, Accuracy = 0.7766667008399963
PERFORMANCE ON TEST SET: Batch Loss = 0.7317416667938232, Accuracy = 0.6247168183326721
Training iter #810000:   Batch Loss = 0.703408, Accuracy = 0.6399999856948853
PERFORMANCE ON TEST SET: Batch Loss = 0.7187536358833313, Accuracy = 0.6247168779373169
Training iter #825000:   Batch Loss = 0.649497, Accuracy = 0.7266666889190674
PERFORMANCE ON TEST SET: Batch Loss = 0.7275341153144836, Accuracy = 0.6214447617530823
Train

PERFORMANCE ON TEST SET: Batch Loss = 0.654969334602356, Accuracy = 0.6267304420471191
Training iter #1500000:   Batch Loss = 0.605393, Accuracy = 0.7233333587646484
PERFORMANCE ON TEST SET: Batch Loss = 0.6523698568344116, Accuracy = 0.6425874829292297
Training iter #1515000:   Batch Loss = 0.726304, Accuracy = 0.5633333325386047
PERFORMANCE ON TEST SET: Batch Loss = 0.6743023991584778, Accuracy = 0.6201862692832947
Training iter #1530000:   Batch Loss = 0.623409, Accuracy = 0.6633332967758179
PERFORMANCE ON TEST SET: Batch Loss = 0.6571059823036194, Accuracy = 0.6244651675224304
Training iter #1545000:   Batch Loss = 0.626497, Accuracy = 0.6566666960716248
PERFORMANCE ON TEST SET: Batch Loss = 0.6534209847450256, Accuracy = 0.6536622643470764
Training iter #1560000:   Batch Loss = 0.497790, Accuracy = 0.7866666316986084
PERFORMANCE ON TEST SET: Batch Loss = 0.6617143750190735, Accuracy = 0.6385602951049805
Training iter #1575000:   Batch Loss = 0.460384, Accuracy = 0.8233333230018616

Training iter #2235000:   Batch Loss = 0.641309, Accuracy = 0.6399999856948853
PERFORMANCE ON TEST SET: Batch Loss = 0.6327468156814575, Accuracy = 0.6722880601882935
Training iter #2250000:   Batch Loss = 0.596862, Accuracy = 0.7199999690055847
PERFORMANCE ON TEST SET: Batch Loss = 0.6488665342330933, Accuracy = 0.648628294467926
Training iter #2265000:   Batch Loss = 0.565466, Accuracy = 0.7599999904632568
PERFORMANCE ON TEST SET: Batch Loss = 0.6292791962623596, Accuracy = 0.6670023202896118
Training iter #2280000:   Batch Loss = 0.615657, Accuracy = 0.6799999475479126
PERFORMANCE ON TEST SET: Batch Loss = 0.6495817303657532, Accuracy = 0.6305059194564819
Training iter #2295000:   Batch Loss = 0.605275, Accuracy = 0.70333331823349
PERFORMANCE ON TEST SET: Batch Loss = 0.6454575061798096, Accuracy = 0.6607098579406738
Training iter #2310000:   Batch Loss = 0.541988, Accuracy = 0.7966667413711548
PERFORMANCE ON TEST SET: Batch Loss = 0.6483161449432373, Accuracy = 0.6513969898223877
T

PERFORMANCE ON TEST SET: Batch Loss = 0.6290846467018127, Accuracy = 0.6702743768692017
Training iter #2985000:   Batch Loss = 0.575712, Accuracy = 0.7099999189376831
PERFORMANCE ON TEST SET: Batch Loss = 0.6278688311576843, Accuracy = 0.6629751920700073
Training iter #3000000:   Batch Loss = 0.560505, Accuracy = 0.7066666483879089
PERFORMANCE ON TEST SET: Batch Loss = 0.6292041540145874, Accuracy = 0.676315188407898
Training iter #3015000:   Batch Loss = 0.623837, Accuracy = 0.6933333277702332
PERFORMANCE ON TEST SET: Batch Loss = 0.6383792161941528, Accuracy = 0.6566826701164246
Training iter #3030000:   Batch Loss = 0.676963, Accuracy = 0.6166666150093079
PERFORMANCE ON TEST SET: Batch Loss = 0.6666487455368042, Accuracy = 0.634029746055603
Training iter #3045000:   Batch Loss = 0.603871, Accuracy = 0.6599999666213989
PERFORMANCE ON TEST SET: Batch Loss = 0.6276256442070007, Accuracy = 0.6629750728607178
Training iter #3060000:   Batch Loss = 0.631066, Accuracy = 0.6666666865348816


Training iter #3720000:   Batch Loss = 0.612679, Accuracy = 0.6833332777023315
PERFORMANCE ON TEST SET: Batch Loss = 0.6311423182487488, Accuracy = 0.6629751324653625
Training iter #3735000:   Batch Loss = 0.646306, Accuracy = 0.6966665983200073
PERFORMANCE ON TEST SET: Batch Loss = 0.6390164494514465, Accuracy = 0.6697710156440735
Training iter #3750000:   Batch Loss = 0.639004, Accuracy = 0.6633332967758179
PERFORMANCE ON TEST SET: Batch Loss = 0.6319790482521057, Accuracy = 0.6685125231742859
Training iter #3765000:   Batch Loss = 0.556074, Accuracy = 0.7433333396911621
PERFORMANCE ON TEST SET: Batch Loss = 0.6265339255332947, Accuracy = 0.6682607531547546
Training iter #3780000:   Batch Loss = 0.719421, Accuracy = 0.6133332848548889
PERFORMANCE ON TEST SET: Batch Loss = 0.6196277141571045, Accuracy = 0.6750566363334656
Optimization Finished!
Model saved in file: /home/soon2soon/Notebooks/DreamChallenge/LSTM-Human-Activity-Recognition/tmp/model.ckpt


In [7]:
sess.close()